In [113]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
fighters_df = pd.read_csv('ufc_fighters_cleaned.csv')

url = "http://ufcstats.com/statistics/events/completed?page=all"
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
response.raise_for_status()
soup = BeautifulSoup(response.content, 'html.parser')

first_tag = soup.select_one('a.b-link.b-link_style_black[href*="event-details"]')
first_event_link = first_tag['href'] if first_tag else None

print(first_event_link)


http://ufcstats.com/event-details/8ad022dd81224f61


In [114]:
fight_links = []

if first_event_link:
    # fetch the single event page
    resp = requests.get(first_event_link, headers=headers)
    resp.raise_for_status()
    ev_soup = BeautifulSoup(resp.content, 'html.parser')

    # extract the event date
    date_el = ev_soup.find('li', class_='b-list__box-list-item')
    event_date = date_el.text.split(':')[-1].strip() if date_el else 'N/A'

    # find every fight row and pull out the onclick URL
    for row in ev_soup.select('tr.b-fight-details__table-row'):
        onclick = row.get('onclick','')
        if 'doNav' in onclick:
            fight_url = onclick.split("'")[1]
            fight_links.append({
                'url': fight_url,
                'event_date': event_date
            })

print(fight_links)


[{'url': 'http://ufcstats.com/fight-details/a81ad236a2f221f2', 'event_date': 'May 17, 2025'}, {'url': 'http://ufcstats.com/fight-details/dcb27eef6441268c', 'event_date': 'May 17, 2025'}, {'url': 'http://ufcstats.com/fight-details/b3044bf646306e7f', 'event_date': 'May 17, 2025'}, {'url': 'http://ufcstats.com/fight-details/fe2acfdaaaec3dba', 'event_date': 'May 17, 2025'}, {'url': 'http://ufcstats.com/fight-details/a9a6cca4bbf7a5db', 'event_date': 'May 17, 2025'}, {'url': 'http://ufcstats.com/fight-details/8ba31cf0b3164b72', 'event_date': 'May 17, 2025'}, {'url': 'http://ufcstats.com/fight-details/d6e2bc6e6c0cc4fe', 'event_date': 'May 17, 2025'}, {'url': 'http://ufcstats.com/fight-details/f75fe8b384b0db11', 'event_date': 'May 17, 2025'}, {'url': 'http://ufcstats.com/fight-details/66887532be2de5ee', 'event_date': 'May 17, 2025'}, {'url': 'http://ufcstats.com/fight-details/18a7dc86279066f8', 'event_date': 'May 17, 2025'}, {'url': 'http://ufcstats.com/fight-details/afdd95ae7b7cdb0c', 'event_

In [115]:
def scrape_ufc_fight_details(url): # scrapes the stats from the url we input
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract method of victory
    method_tag = soup.find('i', class_='b-fight-details__text-item_first')
    method = method_tag.find('i', style="font-style: normal").text.strip() if method_tag else ''
    
    # Extract referee
    referee = ""
    referee_span = soup.select_one('i.b-fight-details__label:-soup-contains("Referee:") + span')
    if referee_span:
        referee = referee_span.text.strip()

    # Find fighter status (W/L) for the first fighter
    fighter_status_tag = soup.find('i', class_='b-fight-details__person-status')
    first_fighter_won = None
    if fighter_status_tag:
        status = fighter_status_tag.text.strip()
        first_fighter_won = 1 if status == "W" else 0
    

    # get the data from round 1 and the totals rows
    all_rows = []
    rows = soup.find_all('tr', class_='b-fight-details__table-row')

    for row in rows:
        columns = row.find_all('p', class_='b-fight-details__table-text')
        data = [col.get_text(strip=True) for col in columns]
        all_rows.append(data)
    
    totals = 1
    r1_totals = 3
    sigTotals = int(len(all_rows) / 2 + 1)
    r1_sigTotals = sigTotals + 2
    
    filtered_rows = []
    
    def is_valid_row(arr):
        needed_indexes = [totals, r1_totals, sigTotals, r1_sigTotals]
        for idx in needed_indexes:
            if idx < len(all_rows):  # Make sure index is valid
                arr.append(all_rows[idx])
    
    is_valid_row(filtered_rows)
    
    # Append method of victory and referee as a new array at the end
    filtered_rows.append([method, referee])
    filtered_rows.append(first_fighter_won)

    return filtered_rows

# Test with the example URL
url = 'http://ufcstats.com/fight-details/d05cb4c4135ce402'
data = scrape_ufc_fight_details(url)
print(data)

[['Michael Chandler', 'Paddy Pimblett', '0', '0', '11 of 28', '80 of 143', '39%', '55%', '20 of 39', '121 of 197', '4 of 7', '1 of 2', '57%', '50%', '0', '0', '0', '1', '2:44', '4:41'], ['Michael Chandler', 'Paddy Pimblett', '0', '0', '5 of 11', '16 of 40', '45%', '40%', '7 of 14', '25 of 50', '2 of 3', '0 of 0', '66%', '---', '0', '0', '0', '0', '2:10', '0:00'], ['Michael Chandler', 'Paddy Pimblett', '11 of 28', '80 of 143', '39%', '55%', '6 of 18', '61 of 119', '3 of 7', '4 of 5', '2 of 3', '15 of 19', '10 of 24', '43 of 96', '0 of 0', '4 of 5', '1 of 4', '33 of 42'], ['Michael Chandler', 'Paddy Pimblett', '5 of 11', '16 of 40', '45%', '40%', '4 of 10', '7 of 28', '1 of 1', '1 of 2', '0 of 0', '8 of 10', '4 of 7', '16 of 40', '0 of 0', '0 of 0', '1 of 4', '0 of 0'], ['KO/TKO', 'Kerry Hatley'], 0]


In [116]:
def format_fight_data_single_row(data): #formats the data extracted from a match into a dictionary
    if not data or len(data) < 4:
        return "Not enough data to format."
    
    # Extract fighter names for reference
    fighter1 = data[0][0]
    fighter2 = data[0][1]
    
    # Create a single row dictionary with p1_ and p2_ prefixes
    formatted_row = {
        "p1_fighter": fighter1,
        "p2_fighter": fighter2,
        
        # Totals (Row 1)
        "p1_KD": data[0][2],
        "p2_KD": data[0][3],
        "p1_SIG_STR": data[0][4],
        "p2_SIG_STR": data[0][5],
        "p1_SIG_STR_PCT": data[0][6],
        "p2_SIG_STR_PCT": data[0][7],
        "p1_TOTAL_STR": data[0][8],
        "p2_TOTAL_STR": data[0][9],
        "p1_TD": data[0][10],
        "p2_TD": data[0][11],
        "p1_TD_PCT": data[0][12],
        "p2_TD_PCT": data[0][13],
        "p1_SUB_ATT": data[0][14],
        "p2_SUB_ATT": data[0][15],
        "p1_REV": data[0][16],
        "p2_REV": data[0][17],
        "p1_CTRL": data[0][18],
        "p2_CTRL": data[0][19],
        
        # Round 1 Totals (Row 2)
        "p1_R1_KD": data[1][2],
        "p2_R1_KD": data[1][3],
        "p1_R1_SIG_STR": data[1][4],
        "p2_R1_SIG_STR": data[1][5],
        "p1_R1_SIG_STR_PCT": data[1][6],
        "p2_R1_SIG_STR_PCT": data[1][7],
        "p1_R1_TOTAL_STR": data[1][8],
        "p2_R1_TOTAL_STR": data[1][9],
        "p1_R1_TD": data[1][10],
        "p2_R1_TD": data[1][11],
        "p1_R1_TD_PCT": data[1][12],
        "p2_R1_TD_PCT": data[1][13],
        "p1_R1_SUB_ATT": data[1][14],
        "p2_R1_SUB_ATT": data[1][15],
        "p1_R1_REV": data[1][16],
        "p2_R1_REV": data[1][17],
        "p1_R1_CTRL": data[1][18],
        "p2_R1_CTRL": data[1][19],
        
        # Significant Strike Details (Row 3)
        "p1_SIG_STR_LANDED_ATTEMPTED": data[2][2],
        "p2_SIG_STR_LANDED_ATTEMPTED": data[2][3],
        "p1_SIG_STR_PCT_DETAILED": data[2][4],
        "p2_SIG_STR_PCT_DETAILED": data[2][5],
        "p1_HEAD_LANDED_ATTEMPTED": data[2][6],
        "p2_HEAD_LANDED_ATTEMPTED": data[2][7],
        "p1_BODY_LANDED_ATTEMPTED": data[2][8],
        "p2_BODY_LANDED_ATTEMPTED": data[2][9],
        "p1_LEG_LANDED_ATTEMPTED": data[2][10],
        "p2_LEG_LANDED_ATTEMPTED": data[2][11],
        "p1_DISTANCE_LANDED_ATTEMPTED": data[2][12],
        "p2_DISTANCE_LANDED_ATTEMPTED": data[2][13],
        "p1_CLINCH_LANDED_ATTEMPTED": data[2][14],
        "p2_CLINCH_LANDED_ATTEMPTED": data[2][15],
        "p1_GROUND_LANDED_ATTEMPTED": data[2][16],
        "p2_GROUND_LANDED_ATTEMPTED": data[2][17],
        
        # Round 1 Significant Strike Details (Row 4)
        "p1_R1_SIG_STR_LANDED_ATTEMPTED": data[3][2],
        "p2_R1_SIG_STR_LANDED_ATTEMPTED": data[3][3],
        "p1_R1_SIG_STR_PCT_DETAILED": data[3][4],
        "p2_R1_SIG_STR_PCT_DETAILED": data[3][5],
        "p1_R1_HEAD_LANDED_ATTEMPTED": data[3][6],
        "p2_R1_HEAD_LANDED_ATTEMPTED": data[3][7],
        "p1_R1_BODY_LANDED_ATTEMPTED": data[3][8],
        "p2_R1_BODY_LANDED_ATTEMPTED": data[3][9],
        "p1_R1_LEG_LANDED_ATTEMPTED": data[3][10],
        "p2_R1_LEG_LANDED_ATTEMPTED": data[3][11],
        "p1_R1_DISTANCE_LANDED_ATTEMPTED": data[3][12],
        "p2_R1_DISTANCE_LANDED_ATTEMPTED": data[3][13],
        "p1_R1_CLINCH_LANDED_ATTEMPTED": data[3][14],
        "p2_R1_CLINCH_LANDED_ATTEMPTED": data[3][15],
        "p1_R1_GROUND_LANDED_ATTEMPTED": data[3][16],
        "p2_R1_GROUND_LANDED_ATTEMPTED": data[3][17],

        "method": data[4][0],
        "referee": data[4][1],
        "winner": data[5]
    }
    
    # Print column headers once (useful for CSV export)
    print(",".join(formatted_row.keys()))
    
    # Print the values as a single row
    print(",".join(str(val) for val in formatted_row.values()))
    
    return formatted_row

# Call the function with your scraped data
formatted_row = format_fight_data_single_row(data)
print(formatted_row)

p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR,p2_SIG_STR,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TOTAL_STR,p2_TOTAL_STR,p1_TD,p2_TD,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR,p2_R1_SIG_STR,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TOTAL_STR,p2_R1_TOTAL_STR,p1_R1_TD,p2_R1_TD,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_LANDED_ATTEMPTED,p2_SIG_STR_LANDED_ATTEMPTED,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_HEAD_LANDED_ATTEMPTED,p2_HEAD_LANDED_ATTEMPTED,p1_BODY_LANDED_ATTEMPTED,p2_BODY_LANDED_ATTEMPTED,p1_LEG_LANDED_ATTEMPTED,p2_LEG_LANDED_ATTEMPTED,p1_DISTANCE_LANDED_ATTEMPTED,p2_DISTANCE_LANDED_ATTEMPTED,p1_CLINCH_LANDED_ATTEMPTED,p2_CLINCH_LANDED_ATTEMPTED,p1_GROUND_LANDED_ATTEMPTED,p2_GROUND_LANDED_ATTEMPTED,p1_R1_SIG_STR_LANDED_ATTEMPTED,p2_R1_SIG_STR_LANDED_ATTEMPTED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,p1_R1_HEAD_LANDED_ATTEMPTED,p2_R1_HEAD_LAN

In [117]:
def process_multiple_fights_with_date(url_date_list): # gets the data from every match and creates a dataframe from it 
    all_fights_data = []
    
    for item in url_date_list:
        url = item['url']
        event_date = item['event_date']
        try:
            fight_data = scrape_ufc_fight_details(url)
            formatted_row = format_fight_data_single_row(fight_data)  
            if isinstance(formatted_row, dict):
                formatted_row['event_date'] = event_date  # Add date to row
                all_fights_data.append(formatted_row)
            else:
                print(f"Error processing {url}: {formatted_row}")
        except Exception as e:
            print(f"Error processing {url}: {str(e)}")
    
    return pd.DataFrame(all_fights_data) if all_fights_data else "No data processed"

fights_df = process_multiple_fights_with_date(fight_links)
fights_df.head()

p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR,p2_SIG_STR,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TOTAL_STR,p2_TOTAL_STR,p1_TD,p2_TD,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR,p2_R1_SIG_STR,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TOTAL_STR,p2_R1_TOTAL_STR,p1_R1_TD,p2_R1_TD,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_LANDED_ATTEMPTED,p2_SIG_STR_LANDED_ATTEMPTED,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_HEAD_LANDED_ATTEMPTED,p2_HEAD_LANDED_ATTEMPTED,p1_BODY_LANDED_ATTEMPTED,p2_BODY_LANDED_ATTEMPTED,p1_LEG_LANDED_ATTEMPTED,p2_LEG_LANDED_ATTEMPTED,p1_DISTANCE_LANDED_ATTEMPTED,p2_DISTANCE_LANDED_ATTEMPTED,p1_CLINCH_LANDED_ATTEMPTED,p2_CLINCH_LANDED_ATTEMPTED,p1_GROUND_LANDED_ATTEMPTED,p2_GROUND_LANDED_ATTEMPTED,p1_R1_SIG_STR_LANDED_ATTEMPTED,p2_R1_SIG_STR_LANDED_ATTEMPTED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,p1_R1_HEAD_LANDED_ATTEMPTED,p2_R1_HEAD_LAN

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR,p2_SIG_STR,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TOTAL_STR,p2_TOTAL_STR,p1_TD,p2_TD,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR,p2_R1_SIG_STR,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TOTAL_STR,p2_R1_TOTAL_STR,p1_R1_TD,p2_R1_TD,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_LANDED_ATTEMPTED,p2_SIG_STR_LANDED_ATTEMPTED,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_HEAD_LANDED_ATTEMPTED,p2_HEAD_LANDED_ATTEMPTED,p1_BODY_LANDED_ATTEMPTED,p2_BODY_LANDED_ATTEMPTED,p1_LEG_LANDED_ATTEMPTED,p2_LEG_LANDED_ATTEMPTED,p1_DISTANCE_LANDED_ATTEMPTED,p2_DISTANCE_LANDED_ATTEMPTED,p1_CLINCH_LANDED_ATTEMPTED,p2_CLINCH_LANDED_ATTEMPTED,p1_GROUND_LANDED_ATTEMPTED,p2_GROUND_LANDED_ATTEMPTED,p1_R1_SIG_STR_LANDED_ATTEMPTED,p2_R1_SIG_STR_LANDED_ATTEMPTED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,p1_R1_HEAD_LANDED_ATTEMPTED,p2_R1_HEAD_LANDED_ATTEMPTED,p1_R1_BODY_LANDED_ATTEMPTED,p2_R1_BODY_LANDED_ATTEMPTED,p1_R1_LEG_LANDED_ATTEMPTED,p2_R1_LEG_LANDED_ATTEMPTED,p1_R1_DISTANCE_LANDED_ATTEMPTED,p2_R1_DISTANCE_LANDED_ATTEMPTED,p1_R1_CLINCH_LANDED_ATTEMPTED,p2_R1_CLINCH_LANDED_ATTEMPTED,p1_R1_GROUND_LANDED_ATTEMPTED,p2_R1_GROUND_LANDED_ATTEMPTED,method,referee,winner,event_date
0,Gilbert Burns,Michael Morales,0,2,5 of 18,33 of 56,27%,58%,5 of 18,35 of 58,1 of 2,0 of 0,50%,---,0,0,0,0,0:30,0:10,0,2,5 of 18,33 of 56,27%,58%,5 of 18,35 of 58,1 of 2,0 of 0,50%,---,0,0,0,0,0:30,0:10,5 of 18,33 of 56,27%,58%,4 of 13,31 of 52,0 of 3,0 of 0,1 of 2,2 of 4,5 of 18,25 of 44,0 of 0,2 of 3,0 of 0,6 of 9,5 of 18,33 of 56,27%,58%,4 of 13,31 of 52,0 of 3,0 of 0,1 of 2,2 of 4,5 of 18,25 of 44,0 of 0,2 of 3,0 of 0,6 of 9,KO/TKO,Herb Dean,0,"May 17, 2025"
1,Sodiq Yusuff,Mairon Santos,0,0,36 of 86,40 of 83,41%,48%,60 of 115,71 of 114,1 of 3,0 of 0,33%,---,0,0,0,0,2:02,1:57,0,0,19 of 30,9 of 27,63%,33%,19 of 30,9 of 27,0 of 1,0 of 0,0%,---,0,0,0,0,0:42,0:03,36 of 86,40 of 83,41%,48%,9 of 44,22 of 56,9 of 19,10 of 18,18 of 23,8 of 9,35 of 84,35 of 77,1 of 2,5 of 6,0 of 0,0 of 0,19 of 30,9 of 27,63%,33%,4 of 11,4 of 18,7 of 10,2 of 5,8 of 9,3 of 4,18 of 29,8 of 26,1 of 1,1 of 1,0 of 0,0 of 0,Decision - Unanimous,Kerry Hatley,0,"May 17, 2025"
2,Dustin Stoltzfus,Nursulton Ruziboev,0,0,21 of 37,35 of 68,56%,51%,41 of 62,53 of 94,2 of 5,1 of 2,40%,50%,2,0,0,2,5:11,3:21,0,0,5 of 13,13 of 27,38%,48%,10 of 18,21 of 36,0 of 1,1 of 1,0%,100%,1,0,0,1,1:14,1:24,21 of 37,35 of 68,56%,51%,5 of 19,24 of 53,4 of 6,8 of 11,12 of 12,3 of 4,19 of 34,24 of 52,1 of 2,1 of 1,1 of 1,10 of 15,5 of 13,13 of 27,38%,48%,2 of 10,8 of 19,1 of 1,2 of 4,2 of 2,3 of 4,5 of 13,11 of 25,0 of 0,1 of 1,0 of 0,1 of 1,Decision - Unanimous,Chris Tognoni,0,"May 17, 2025"
3,Julian Erosa,Melquizael Costa,0,0,69 of 173,102 of 204,39%,50%,85 of 202,128 of 236,3 of 10,2 of 3,30%,66%,0,1,0,0,3:34,0:23,0,0,10 of 26,32 of 75,38%,42%,10 of 27,32 of 75,1 of 3,0 of 0,33%,---,0,0,0,0,0:36,0:01,69 of 173,102 of 204,39%,50%,42 of 140,67 of 160,18 of 24,16 of 22,9 of 9,19 of 22,55 of 150,99 of 196,13 of 21,3 of 7,1 of 2,0 of 1,10 of 26,32 of 75,38%,42%,5 of 19,14 of 52,1 of 3,7 of 10,4 of 4,11 of 13,9 of 24,32 of 73,0 of 0,0 of 2,1 of 2,0 of 0,Decision - Unanimous,Herb Dean,0,"May 17, 2025"
4,Gabe Green,Matheus Camilo,0,0,12 of 27,13 of 27,44%,48%,36 of 57,33 of 48,0 of 0,2 of 3,---,66%,1,1,0,0,2:12,2:30,0,0,7 of 18,12 of 24,38%,50%,17 of 30,30 of 43,0 of 0,1 of 2,---,50%,0,0,0,0,0:09,1:26,12 of 27,13 of 27,44%,48%,4 of 16,6 of 16,3 of 4,4 of 6,5 of 7,3 of 5,10 of 25,12 of 24,1 of 1,0 of 1,1 of 1,1 of 2,7 of 18,12 of 24,38%,50%,1 of 9,6 of 14,1 of 2,4 of 6,5 of 7,2 of 4,6 of 17,11 of 21,1 of 1,0 of 1,0 of 0,1 of 2,Submission,Jason Herzog,1,"May 17, 2025"


In [118]:
#clean fights_df

#replace '--' values and turn percents to decimals
percent_cols = ['p1_SIG_STR_PCT', 'p2_SIG_STR_PCT', 'p1_TD_PCT', 'p2_TD_PCT', 'p1_R1_SIG_STR_PCT', 'p2_R1_SIG_STR_PCT', 'p1_R1_TD_PCT', 'p2_R1_TD_PCT', 'p1_SIG_STR_PCT_DETAILED', 'p2_SIG_STR_PCT_DETAILED', 'p1_R1_SIG_STR_PCT_DETAILED', 'p2_R1_SIG_STR_PCT_DETAILED']
for col in percent_cols:
    fights_df[col] = (
        fights_df[col]
        .astype(str)
        .replace({'--': np.nan, '---': np.nan})  # Handle missing values first
        .str.replace('%', '', regex=False)  # Remove percentage symbols
        .str.strip()  # Clean whitespace
        .astype(float) / 100  # Convert to decimal
    )


#format event date
fights_df['event_date'] = pd.to_datetime(fights_df['event_date'], errors='coerce')


# columns to split
cols_to_split = ['p1_SIG_STR', 'p2_SIG_STR', 'p1_TOTAL_STR', 'p2_TOTAL_STR', 'p1_TD', 'p2_TD', 'p1_R1_SIG_STR', 'p2_R1_SIG_STR', 'p1_R1_TOTAL_STR', 'p2_R1_TOTAL_STR', 'p1_R1_TD', 'p2_R1_TD', 'p1_SIG_STR_LANDED_ATTEMPTED', 'p2_SIG_STR_LANDED_ATTEMPTED', 'p1_HEAD_LANDED_ATTEMPTED', 'p2_HEAD_LANDED_ATTEMPTED', 'p1_BODY_LANDED_ATTEMPTED', 'p2_BODY_LANDED_ATTEMPTED', 'p1_LEG_LANDED_ATTEMPTED', 'p2_LEG_LANDED_ATTEMPTED', 'p1_DISTANCE_LANDED_ATTEMPTED', 'p2_DISTANCE_LANDED_ATTEMPTED', 'p1_CLINCH_LANDED_ATTEMPTED', 'p2_CLINCH_LANDED_ATTEMPTED', 'p1_GROUND_LANDED_ATTEMPTED', 'p2_GROUND_LANDED_ATTEMPTED', 'p1_R1_SIG_STR_LANDED_ATTEMPTED', 'p2_R1_SIG_STR_LANDED_ATTEMPTED', 'p1_R1_HEAD_LANDED_ATTEMPTED', 'p2_R1_HEAD_LANDED_ATTEMPTED', 'p1_R1_BODY_LANDED_ATTEMPTED', 'p2_R1_BODY_LANDED_ATTEMPTED', 'p1_R1_LEG_LANDED_ATTEMPTED', 'p2_R1_LEG_LANDED_ATTEMPTED', 'p1_R1_DISTANCE_LANDED_ATTEMPTED', 'p2_R1_DISTANCE_LANDED_ATTEMPTED', 'p1_R1_CLINCH_LANDED_ATTEMPTED', 'p2_R1_CLINCH_LANDED_ATTEMPTED', 'p1_R1_GROUND_LANDED_ATTEMPTED', 'p2_R1_GROUND_LANDED_ATTEMPTED']

#Loop over them, split into “_LANDED” and “_ATTEMPTED”, convert to ints
for col in cols_to_split:
    # decide new column names
    if col.endswith('_LANDED_ATTEMPTED'):
        base = col.rsplit('_', 2)[0]       # turns 'p1_HEAD_LANDED_ATTEMPTED' → 'p1_HEAD'
        landed_col    = f"{base}_LANDED"
        attempted_col = f"{base}_ATTEMPTED"
    else:
        # for columns like 'p1_SIG_STR'
        landed_col    = f"{col}_LANDED"
        attempted_col = f"{col}_ATTEMPTED"

    # split “63 of 129” → ['63','129'], then to numeric
    fights_df[[landed_col, attempted_col]] = (
        fights_df[col]
          .str.split(' of ', expand=True)
          .astype(int)
    )

# Drop the originals
fights_df.drop(columns=cols_to_split, inplace=True)


# time collumns needing reformatting
time_cols = ['p1_CTRL', 'p2_CTRL', 'p1_R1_CTRL', 'p2_R1_CTRL' ]

#Convert each to total seconds
for col in time_cols:
    # Replace non-time strings with NaN
    fights_df[col] = fights_df[col].replace({'--': np.nan, '---': np.nan})
    
    # Convert to "00:MM:SS" format then to seconds
    fights_df[col] = (
        pd.to_timedelta('00:' + fights_df[col].fillna('00:00'), errors='coerce')
        .dt.total_seconds()
        .astype('Int64')  # Allows integer NaNs
    )

fights_df.head(10)

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED
0,Gilbert Burns,Michael Morales,0,2,0.27,0.58,0.50,NaN,0,0,0,0,30,10,0,2,0.27,0.58,0.50,NaN,0,0,0,0,30,10,0.27,0.58,0.27,0.58,KO/TKO,Herb Dean,0,2025-05-17,5,18,33,56,5,18,35,58,1,2,0,0,5,18,33,56,5,18,35,58,1,2,0,0,4,13,31,52,0,3,0,0,1,2,2,4,5,18,25,44,0,0,2,3,0,0,6,9,4,13,31,52,0,3,0,0,1,2,2,4,5,18,25,44,0,0,2,3,0,0,6,9
1,Sodiq Yusuff,Mairon Santos,0,0,0.41,0.48,0.33,NaN,0,0,0,0,122,117,0,0,0.63,0.33,0.00,NaN,0,0,0,0,42,3,0.41,0.48,0.63,0.33,Decision - Unanimous,Kerry Hatley,0,2025-05-17,36,86,40,83,60,115,71,114,1,3,0,0,19,30,9,27,19,30,9,27,0,1,0,0,9,44,22,56,9,19,10,18,18,23,8,9,35,84,35,77,1,2,5,6,0,0,0,0,4,11,4,18,7,10,2,5,8,9,3,4,18,29,8,26,1,1,1,1,0,0,0,0
2,Dustin Stoltzfus,Nursulton Ruziboev,0,0,0.56,0.51,0.40,0.50,2,0,0,2,311,201,0,0,0.38,0.48,0.00,1.0,1,0,0,1,74,84,0.56,0.51,0.38,0.48,Decision - Unanimous,Chris Tognoni,0,2025-05-17,21,37,35,68,41,62,53,94,2,5,1,2,5,13,13,27,10,18,21,36,0,1,1,1,5,19,24,53,4,6,8,11,12,12,3,4,19,34,24,52,1,2,1,1,1,1,10,15,2,10,8,19,1,1,2,4,2,2,3,4,5,13,11,25,0,0,1,1,0,0,1,1
3,Julian Erosa,Melquizael Costa,0,0,0.39,0.50,0.30,0.66,0,1,0,0,214,23,0,0,0.38,0.42,0.33,NaN,0,0,0,0,36,1,0.39,0.50,0.38,0.42,Decision - Unanimous,Herb Dean,0,2025-05-17,69,173,102,204,85,202,128,236,3,10,2,3,10,26,32,75,10,27,32,75,1,3,0,0,42,140,67,160,18,24,16,22,9,9,19,22,55,150,99,196,13,21,3,7,1,2,0,1,5,19,14,52,1,3,7,10,4,4,11,13,9,24,32,73,0,0,0,2,1,2,0,0
4,Gabe Green,Matheus Camilo,0,0,0.44,0.48,NaN,0.66,1,1,0,0,132,150,0,0,0.38,0.50,NaN,0.5,0,0,0,0,9,86,0.44,0.48,0.38,0.50,Submission,Jason Herzog,1,2025-05-17,12,27,13,27,36,57,33,48,0,0,2,3,7,18,12,24,17,30,30,43,0,0,1,2,4,16,6,16,3,4,4,6,5,7,3,5,10,25,12,24,1,1,0,1,1,1,1,2,1,9,6,14,1,2,4,6,5,7,2,4,6,17,11,21,1,1,0,1,0,0,1,2
5,Jared Gordon,Thiago Moises,1,0,0.53,0.55,1.00,NaN,0,0,0,0,156,0,1,0,0.53,0.55,1.00,NaN,0,0,0,0,156,0,0.53,0.55,0.53,0.55,KO/TKO,Kerry Hatley,1,2025-05-17,7,13,5,9,21,36,11,16,1,1,0,0,7,13,5,9,21,36,11,16,1,1,0,0,5,10,3,6,0,0,0,1,2,3,2,2,5,11,5,9,0,0,0,0,2,2,0,0,5,10,3,6,0,0,0,1,2,3,2,2,5,11,5,9,0,0,0,0,2,2,0,0
6,Yadier del Valle,Connor Matthews,0,0,0.75,0.30,1.00,NaN,1,0,0,0,81,0,0,0,0.75,0.30,1.00,NaN,1,0,0,0,81,0,0.75,0.30,0.75,0.30,Submission

In [119]:
# merge the two datasets

# First, merge p1 fighter stats
fights_df = fights_df.merge(
    fighters_df,
    left_on='p1_fighter',
    right_on='name',  # This matches p1_fighter with the name column in fighters_df
    how='left'
)
fights_df.drop(columns=['name'], inplace=True)

# Only include the specific columns you want to rename with p1_ prefix
p1_cols = ['height', 'weight', 'reach', 'stance', 'dob', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.']
fights_df.rename(columns={col: f'p1_{col}' for col in p1_cols}, inplace=True)

# Then merge p2 fighter stats
fights_df = fights_df.merge(
    fighters_df,
    left_on='p2_fighter',
    right_on='name',  # This matches p2_fighter with the name column in fighters_df
    how='left'
)
fights_df.drop(columns=['name'], inplace=True)

# Use the same columns for p2 as we did for p1
p2_cols = ['height', 'weight', 'reach', 'stance', 'dob', 'SLpM', 'Str. Acc.', 'SApM', 'Str. Def', 'TD Avg.', 'TD Acc.', 'TD Def.', 'Sub. Avg.']
fights_df.rename(columns={col: f'p2_{col}' for col in p2_cols}, inplace=True)


# Drop unwanted columns
columns_to_drop = ['record_x', 'wins_x', 'losses_x', 'draws_x', 'nc_x', 'record_y', 'wins_y', 'losses_y', 'draws_y', 'nc_y', 'url_x', 'url_y']

# Check if columns exist before trying to drop them
existing_columns = [col for col in columns_to_drop if col in fights_df.columns]
if existing_columns:
    fights_df.drop(columns=existing_columns, inplace=True)


#-------------------------
# cleaned_df = fights_df      # update the cleaned csv so that we cna retarin the features_df to update the model and calculating ema
#-------------------------


fights_df.head()

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED,p1_height,p1_weight,p1_reach,p1_stance,p1_dob,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_dob,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.
0,Gilbert Burns,Michael Morales,0,2,0.27,0.58,0.50,NaN,0,0,0,0,30,10,0,2,0.27,0.58,0.50,NaN,0,0,0,0,30,10,0.27,0.58,0.27,0.58,KO/TKO,Herb Dean,0,2025-05-17,5,18,33,56,5,18,35,58,1,2,0,0,5,18,33,56,5,18,35,58,1,2,0,0,4,13,31,52,0,3,0,0,1,2,2,4,5,18,25,44,0,0,2,3,0,0,6,9,4,13,31,52,0,3,0,0,1,2,2,4,5,18,25,44,0,0,2,3,0,0,6,9,70.0,170.0,71.0,Orthodox,1986-07-20,3.17,0.48,3.57,0.52,2.10,0.37,0.53,0.5,72.0,170.0,79.0,Orthodox,1999-06-24,5.42,0.48,3.42,0.52,1.16,0.41,0.92,0.0
1,Sodiq Yusuff,Mairon Santos,0,0,0.41,0.48,0.33,NaN,0,0,0,0,122,117,0,0,0.63,0.33,0.00,NaN,0,0,0,0,42,3,0.41,0.48,0.63,0.33,Decision - Unanimous,Kerry Hatley,0,2025-05-17,36,86,40,83,60,115,71,114,1,3,0,0,19,30,9,27,19,30,9,27,0,1,0,0,9,44,22,56,9,19,10,18,18,23,8,9,35,84,35,77,1,2,5,6,0,0,0,0,4,11,4,18,7,10,2,5,8,9,3,4,18,29,8,26,1,1,1,1,0,0,0,0,69.0,155.0,71.0,Orthodox,1993-05-19,5.66,0.49,4.51,0.53,0.28,0.20,0.62,0.4,67.0,155.0,72.0,Orthodox,2000-06-10,4.14,0.45,2.37,0.70,0.00,0.00,0.40,0.0
2,Dustin Stoltzfus,Nursulton Ruziboev,0,0,0.56,0.51,0.40,0.50,2,0,0,2,311,201,0,0,0.38,0.48,0.00,1.0,1,0,0,1,74,84,0.56,0.51,0.38,0.48,Decision - Unanimous,Chris Tognoni,0,2025-05-17,21,37,35,68,41,62,53,94,2,5,1,2,5,13,13,27,10,18,21,36,0,1,1,1,5,19,24,53,4,6,8,11,12,12,3,4,19,34,24,52,1,2,1,1,1,1,10,15,2,10,8,19,1,1,2,4,2,2,3,4,5,13,11,25,0,0,1,1,0,0,1,1,72.0,185.0,75.0,Orthodox,1991-11-15,3.47,0.44,3.29,0.46,2.31,0.41,0.46,1.1,77.0,185.0,76.0,Orthodox,1993-11-19,2.75,0.44,3.78,0.50,0.60,1.00,0.00,0.0
3,Julian Erosa,Melquizael Costa,0,0,0.39,0.50,0.30,0.66,0,1,0,0,214,23,0,0,0.38,0.42,0.33,NaN,0,0,0,0,36,1,0.39,0.50,0.38,0.42,Decision - Unanimous,Herb Dean,0,2025-05-17,69,173,102,204,85,202,128,236,3,10,2,3,10,26,32,75,10,27,32,75,1,3,0,0,42,140,67,160,18,24,16,22,9,9,19,22,55,150,99,196,13,21,3,7,1,2,0,1,5,19,14,52,1,3,7,10,4,4,11,13,9,24,32,73,0,0,0,2,1,2,0,0,73.0,145.0,74.0,Southpaw,1989-07-31,6.40,0.49,6.21,0.48,1.63,0.

In [120]:
import pandas as pd
import numpy as np
from collections import defaultdict

# --- 1) Load data ---
cleaned_df = pd.read_csv('ufc_cleaned.csv')

# --- 2) Parse all date columns to datetime ---
cleaned_df['event_date']    = pd.to_datetime(cleaned_df['event_date'],    errors='coerce')
fights_df ['event_date']    = pd.to_datetime(fights_df ['event_date'],    errors='coerce')
fights_df ['p1_dob']        = pd.to_datetime(fights_df ['p1_dob'],        errors='coerce')
fights_df ['p2_dob']        = pd.to_datetime(fights_df ['p2_dob'],        errors='coerce')

# --- 3) Age at event and drop DOBs ---
fights_df['p1_age_at_event'] = (fights_df['event_date'] - fights_df['p1_dob']).dt.days / 365.25
fights_df['p2_age_at_event'] = (fights_df['event_date'] - fights_df['p2_dob']).dt.days / 365.25
fights_df.drop(columns=['p1_dob','p2_dob'], inplace=True)

# --- 4) Physical differences ---
fights_df['height_diff'] = fights_df['p1_height'] - fights_df['p2_height']
fights_df['reach_diff']  = fights_df['p1_reach']  - fights_df['p2_reach']
fights_df['weight_diff'] = fights_df['p1_weight'] - fights_df['p2_weight']
fights_df['age_diff']    = fights_df['p1_age_at_event'] - fights_df['p2_age_at_event']

# --- 5) Fighting‐metric differences ---
metrics = ['SLpM','Str. Acc.','SApM','Str. Def','TD Avg.','TD Acc.','TD Def.','Sub. Avg.']
for m in metrics:
    p1_col   = f'p1_{m}'
    p2_col   = f'p2_{m}'
    diff_col = f'{m.replace(" ", "").replace(".", "").lower()}_diff'
    fights_df[diff_col] = fights_df[p1_col] - fights_df[p2_col]

# --- 6) Age-adjust those same metrics ---
p1_cols = [f'p1_{m}' for m in metrics]
p2_cols = [f'p2_{m}' for m in metrics]
for p1_col, p2_col in zip(p1_cols, p2_cols):
    fights_df[p1_col.replace('p1_','p1_age_adjusted_')] = fights_df[p1_col] / fights_df['p1_age_at_event']
    fights_df[p2_col.replace('p2_','p2_age_adjusted_')] = fights_df[p2_col] / fights_df['p2_age_at_event']

# --- 7) SEED FIGHTER HISTORY FOR RECORD & STREAK ---
# Detect your winner‐encoding once (0→P1, 1→P2)
p1_code, p2_code = 1, 0

# Build a single long table of (fighter, event_date, win_flag)
p1_hist = (
    cleaned_df[['p1_fighter','event_date','winner']]
      .rename(columns={'p1_fighter':'fighter'})
)
p1_hist['win_flag'] = (p1_hist['winner'] == p1_code).astype(int)
p2_hist = (
    cleaned_df[['p2_fighter','event_date','winner']]
      .rename(columns={'p2_fighter':'fighter'})
)
p2_hist['win_flag'] = (p2_hist['winner'] == p2_code).astype(int)

hist = pd.concat([p1_hist[['fighter','event_date','win_flag']],
                  p2_hist[['fighter','event_date','win_flag']]],
                 ignore_index=True)

# 7a) last_date, wins, losses
last_date = hist.groupby('fighter').event_date.max().to_dict()
wins      = hist[hist.win_flag==1].fighter.value_counts().to_dict()
losses    = hist[hist.win_flag==0].fighter.value_counts().to_dict()

# record[f] = (wins, losses, total); default (0,0,0)
init_rec = {f:(wins.get(f,0), losses.get(f,0),
               wins.get(f,0)+losses.get(f,0))
            for f in hist.fighter.unique()}
record     = defaultdict(lambda: (0,0,0), init_rec)

# 7b) current win streak
win_streak = defaultdict(int)
for fighter, grp in hist.sort_values('event_date').groupby('fighter'):
    streak = 0
    for wf in grp.win_flag:
        streak = streak+1 if wf==1 else 0
    win_streak[fighter] = streak

# --- 8) IN‐PLACE RECORD & STREAK LOOP ---
feat_cols = [
  'p1_days_since_last','p2_days_since_last','days_since_diff',
  'p1_wins','p1_losses','p1_total',
  'p2_wins','p2_losses','p2_total',
  'win_diff','loss_diff','total_diff',
  'p1_win_streak','p2_win_streak'
]
for col in feat_cols:
    fights_df[col] = np.nan

fights_df.sort_values('event_date', inplace=True)
for idx, row in fights_df.iterrows():
    p1, p2    = row['p1_fighter'], row['p2_fighter']
    date      = row['event_date']
    winner    = row['winner']

    # days‐since‐last‐fight
    ld1, ld2 = last_date.get(p1), last_date.get(p2)
    d1 = (date - ld1).days if ld1 is not None else np.nan
    d2 = (date - ld2).days if ld2 is not None else np.nan
    fights_df.at[idx, 'p1_days_since_last'] = d1
    fights_df.at[idx, 'p2_days_since_last'] = d2
    fights_df.at[idx, 'days_since_diff']     = d1 - d2

    # pre‐fight records
    w1,l1,t1 = record[p1]
    w2,l2,t2 = record[p2]
    fights_df.at[idx, 'p1_wins']   = w1
    fights_df.at[idx, 'p1_losses'] = l1
    fights_df.at[idx, 'p1_total']  = t1
    fights_df.at[idx, 'p2_wins']   = w2
    fights_df.at[idx, 'p2_losses'] = l2
    fights_df.at[idx, 'p2_total']  = t2
    fights_df.at[idx, 'win_diff']   = w1 - w2
    fights_df.at[idx, 'loss_diff']  = l1 - l2
    fights_df.at[idx, 'total_diff'] = t1 - t2

    # pre‐fight win streaks
    fights_df.at[idx, 'p1_win_streak'] = win_streak[p1]
    fights_df.at[idx, 'p2_win_streak'] = win_streak[p2]

    # update last_date
    last_date[p1] = last_date[p2] = date

    # UPDATE RECORD & STREAK BASED ON YOUR 0/1 ENCODING
    if winner == p1_code:
        # P1 won
        record[p1] = (w1+1, l1,   t1+1)
        record[p2] = (w2,   l2+1, t2+1)
        win_streak[p1] += 1
        win_streak[p2]  = 0
    elif winner == p2_code:
        # P2 won
        record[p1] = (w1,   l1+1, t1+1)
        record[p2] = (w2+1, l2,   t2+1)
        win_streak[p1]  = 0
        win_streak[p2] += 1
    else:
        # unexpected code
        raise ValueError(f"Unknown winner code {winner} at row {idx}")
# --- 9) Compute 3-fight EMA for all stats and merge in place ---
# List of stats in cleaned_df to smooth
features = [
    'KD','SIG_STR_PCT','TD_PCT','SUB_ATT','REV','CTRL',
    'R1_KD','R1_SIG_STR_PCT','R1_TD_PCT','R1_SUB_ATT','R1_REV','R1_CTRL',
    'SIG_STR_PCT_DETAILED','R1_SIG_STR_PCT_DETAILED',
    'SIG_STR_LANDED','SIG_STR_ATTEMPTED','TOTAL_STR_LANDED','TOTAL_STR_ATTEMPTED',
    'TD_LANDED','TD_ATTEMPTED',
    'R1_SIG_STR_LANDED','R1_SIG_STR_ATTEMPTED','R1_TOTAL_STR_LANDED','R1_TOTAL_STR_ATTEMPTED',
    'R1_TD_LANDED','R1_TD_ATTEMPTED',
    'HEAD_LANDED','HEAD_ATTEMPTED','BODY_LANDED','BODY_ATTEMPTED',
    'LEG_LANDED','LEG_ATTEMPTED',
    'DISTANCE_LANDED','DISTANCE_ATTEMPTED','CLINCH_LANDED','CLINCH_ATTEMPTED',
    'GROUND_LANDED','GROUND_ATTEMPTED',
    'R1_HEAD_LANDED','R1_HEAD_ATTEMPTED','R1_BODY_LANDED','R1_BODY_ATTEMPTED',
    'R1_LEG_LANDED','R1_LEG_ATTEMPTED',
    'R1_DISTANCE_LANDED','R1_DISTANCE_ATTEMPTED','R1_CLINCH_LANDED','R1_CLINCH_ATTEMPTED',
    'R1_GROUND_LANDED','R1_GROUND_ATTEMPTED'
]

# Build long history
p1_map = {f'p1_{feat}': feat for feat in features}
p2_map = {f'p2_{feat}': feat for feat in features}

p1_stats = cleaned_df[['p1_fighter','event_date'] + list(p1_map.keys())].rename(columns=p1_map)
p1_stats['fighter']  = cleaned_df['p1_fighter']
p2_stats = cleaned_df[['p2_fighter','event_date'] + list(p2_map.keys())].rename(columns=p2_map)
p2_stats['fighter']  = cleaned_df['p2_fighter']

cols = ['fighter','event_date'] + features
fighter_stats = pd.concat([p1_stats[cols], p2_stats[cols]], ignore_index=True)
fighter_stats.sort_values(['fighter','event_date'], inplace=True)

# numeric cast + EMA
for feat in features:
    fighter_stats[feat] = pd.to_numeric(fighter_stats[feat], errors='coerce')
    fighter_stats[f'{feat}_EMA'] = (
        fighter_stats
        .groupby('fighter')[feat]
        .transform(lambda x: x.ewm(span=3).mean())
    )

# Pre-allocate EMA columns in fights_df
for feat in features:
    fights_df[f'p1_{feat}_EMA'] = np.nan
    fights_df[f'p2_{feat}_EMA'] = np.nan

# Fill EMAs by looking up last available date
fights_df.sort_values('event_date', inplace=True)
for idx, row in fights_df.iterrows():
    date = row['event_date']
    for side in ('p1','p2'):
        f = row[f'{side}_fighter']
        prev = fighter_stats[
            (fighter_stats.fighter   == f) &
            (fighter_stats.event_date < date)
        ]
        if not prev.empty:
            latest = prev.iloc[-1]
            for feat in features:
                fights_df.at[idx, f'{side}_{feat}_EMA'] = latest[f'{feat}_EMA']

# fights_df now contains all new features in place


/var/folders/nc/04zwxn450f3cllk4tzq8_fp40000gn/T/ipykernel_1003/1457797522.py:176: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fighter_stats[f'{feat}_EMA'] = (
/var/folders/nc/04zwxn450f3cllk4tzq8_fp40000gn/T/ipykernel_1003/1457797522.py:176: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fighter_stats[f'{feat}_EMA'] = (
/var/folders/nc/04zwxn450f3cllk4tzq8_fp40000gn/T/ipykernel_1003/1457797522.py:184: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

In [123]:
methods_df        = cleaned_df.copy()     # historical
create_methods_df = fights_df.copy()      # new

# --- 1) Winner codes: 1 means p1 won, 0 means p2 won ---
p1_code, p2_code = 1, 0

# --- 2) Group raw 'method' into four buckets ---
def group_methods(df):
    replace_map = {
        'Decision - Majority':    'Decision',
        'Decision - Split':       'Decision',
        'Decision - Unanimous':   'Decision',
        "TKO - Doctor's Stoppage":'KO/TKO',
        'Overturned':             'Other',
        'Could Not Continue':     'Other',
        'DQ':                     'Other',
        'Other':                  'Other'
    }
    df['method_grouped'] = df['method'].replace(replace_map)
    return df

methods_df        = group_methods(methods_df)
create_methods_df = group_methods(create_methods_df)

# --- 3) Sort historical by date so we seed method_counts correctly ---
methods_df.sort_values('event_date', inplace=True)
methods_df.reset_index(drop=True, inplace=True)

# --- 4) Prepare counters for each method, defaulting to zero for debut fighters ---
method_types = ['Decision','KO/TKO','Submission','Other']
method_counts = {m: defaultdict(int) for m in method_types}

# --- 5) Seed counts from every historical fight in cleaned_df ---
for _, row in methods_df.iterrows():
    mtd    = row['method_grouped']
    winner = row['winner']
    p1     = row['p1_fighter']
    p2     = row['p2_fighter']
    
    if winner == p1_code:
        method_counts[mtd][p1] += 1
    elif winner == p2_code:
        method_counts[mtd][p2] += 1
    # else: ignore unexpected codes

# --- 6) Prepare new-fights DataFrame ---
create_methods_df.sort_values('event_date', inplace=True)
create_methods_df.reset_index(drop=True, inplace=True)

# add the eight result columns
for m in method_types:
    create_methods_df[f'p1_{m}_wins'] = 0
    create_methods_df[f'p2_{m}_wins'] = 0

# --- 7) Walk through the new fights, write pre-fight counts, then update ---
for idx, row in create_methods_df.iterrows():
    p1     = row['p1_fighter']
    p2     = row['p2_fighter']
    winner = row['winner']
    mtd    = row['method_grouped']
    
    # write pre-fight totals
    for m in method_types:
        create_methods_df.at[idx, f'p1_{m}_wins'] = method_counts[m][p1]
        create_methods_df.at[idx, f'p2_{m}_wins'] = method_counts[m][p2]
    
    # then update so the next new fight sees this result
    if winner == p1_code:
        method_counts[mtd][p1] += 1
    elif winner == p2_code:
        method_counts[mtd][p2] += 1

# Result: create_methods_df now has correct p1_*/p2_*_wins for each method

# Display the first few rows to verify
create_methods_df.head()

,p1_fighter,p2_fighter,p1_KD,p2_KD,p1_SIG_STR_PCT,p2_SIG_STR_PCT,p1_TD_PCT,p2_TD_PCT,p1_SUB_ATT,p2_SUB_ATT,p1_REV,p2_REV,p1_CTRL,p2_CTRL,p1_R1_KD,p2_R1_KD,p1_R1_SIG_STR_PCT,p2_R1_SIG_STR_PCT,p1_R1_TD_PCT,p2_R1_TD_PCT,p1_R1_SUB_ATT,p2_R1_SUB_ATT,p1_R1_REV,p2_R1_REV,p1_R1_CTRL,p2_R1_CTRL,p1_SIG_STR_PCT_DETAILED,p2_SIG_STR_PCT_DETAILED,p1_R1_SIG_STR_PCT_DETAILED,p2_R1_SIG_STR_PCT_DETAILED,method,referee,winner,event_date,p1_SIG_STR_LANDED,p1_SIG_STR_ATTEMPTED,p2_SIG_STR_LANDED,p2_SIG_STR_ATTEMPTED,p1_TOTAL_STR_LANDED,p1_TOTAL_STR_ATTEMPTED,p2_TOTAL_STR_LANDED,p2_TOTAL_STR_ATTEMPTED,p1_TD_LANDED,p1_TD_ATTEMPTED,p2_TD_LANDED,p2_TD_ATTEMPTED,p1_R1_SIG_STR_LANDED,p1_R1_SIG_STR_ATTEMPTED,p2_R1_SIG_STR_LANDED,p2_R1_SIG_STR_ATTEMPTED,p1_R1_TOTAL_STR_LANDED,p1_R1_TOTAL_STR_ATTEMPTED,p2_R1_TOTAL_STR_LANDED,p2_R1_TOTAL_STR_ATTEMPTED,p1_R1_TD_LANDED,p1_R1_TD_ATTEMPTED,p2_R1_TD_LANDED,p2_R1_TD_ATTEMPTED,p1_HEAD_LANDED,p1_HEAD_ATTEMPTED,p2_HEAD_LANDED,p2_HEAD_ATTEMPTED,p1_BODY_LANDED,p1_BODY_ATTEMPTED,p2_BODY_LANDED,p2_BODY_ATTEMPTED,p1_LEG_LANDED,p1_LEG_ATTEMPTED,p2_LEG_LANDED,p2_LEG_ATTEMPTED,p1_DISTANCE_LANDED,p1_DISTANCE_ATTEMPTED,p2_DISTANCE_LANDED,p2_DISTANCE_ATTEMPTED,p1_CLINCH_LANDED,p1_CLINCH_ATTEMPTED,p2_CLINCH_LANDED,p2_CLINCH_ATTEMPTED,p1_GROUND_LANDED,p1_GROUND_ATTEMPTED,p2_GROUND_LANDED,p2_GROUND_ATTEMPTED,p1_R1_HEAD_LANDED,p1_R1_HEAD_ATTEMPTED,p2_R1_HEAD_LANDED,p2_R1_HEAD_ATTEMPTED,p1_R1_BODY_LANDED,p1_R1_BODY_ATTEMPTED,p2_R1_BODY_LANDED,p2_R1_BODY_ATTEMPTED,p1_R1_LEG_LANDED,p1_R1_LEG_ATTEMPTED,p2_R1_LEG_LANDED,p2_R1_LEG_ATTEMPTED,p1_R1_DISTANCE_LANDED,p1_R1_DISTANCE_ATTEMPTED,p2_R1_DISTANCE_LANDED,p2_R1_DISTANCE_ATTEMPTED,p1_R1_CLINCH_LANDED,p1_R1_CLINCH_ATTEMPTED,p2_R1_CLINCH_LANDED,p2_R1_CLINCH_ATTEMPTED,p1_R1_GROUND_LANDED,p1_R1_GROUND_ATTEMPTED,p2_R1_GROUND_LANDED,p2_R1_GROUND_ATTEMPTED,p1_height,p1_weight,p1_reach,p1_stance,p1_SLpM,p1_Str. Acc.,p1_SApM,p1_Str. Def,p1_TD Avg.,p1_TD Acc.,p1_TD Def.,p1_Sub. Avg.,p2_height,p2_weight,p2_reach,p2_stance,p2_SLpM,p2_Str. Acc.,p2_SApM,p2_Str. Def,p2_TD Avg.,p2_TD Acc.,p2_TD Def.,p2_Sub. Avg.,p1_age_at_event,p2_age_at_event,height_diff,reach_diff,weight_diff,age_diff,slpm_diff,stracc_diff,sapm_diff,strdef_diff,tdavg_diff,tdacc_diff,tddef_diff,subavg_diff,p1_age_adjusted_SLpM,p2_age_adjusted_SLpM,p1_age_adjusted_Str. Acc.,p2_age_adjusted_Str. Acc.,p1_age_adjusted_SApM,p2_age_adjusted_SApM,p1_age_adjusted_Str. Def,p2_age_adjusted_Str. Def,p1_age_adjusted_TD Avg.,p2_age_adjusted_TD Avg.,p1_age_adjusted_TD Acc.,p2_age_adjusted_TD Acc.,p1_age_adjusted_TD Def.,p2_age_adjusted_TD Def.,p1_age_adjusted_Sub. Avg.,p2_age_adjusted_Sub. Avg.,p1_days_since_last,p2_days_since_last,days_since_diff,p1_wins,p1_losses,p1_total,p2_wins,p2_losses,p2_total,win_diff,loss_diff,total_diff,p1_win_streak,p2_win_streak,p1_KD_EMA,p2_KD_EMA,p1_SIG_STR_PCT_EMA,p2_SIG_STR_PCT_EMA,p1_TD_PCT_EMA,p2_TD_PCT_EMA,p1_SUB_ATT_EMA,p2_SUB_ATT_EMA,p1_REV_EMA,p2_REV_EMA,p1_CTRL_EMA,p2_CTRL_EMA,p1_R1_KD_EMA,p2_R1_KD_EMA,p1_R1_SIG_STR_PCT_EMA,p2_R1_SIG_STR_PCT_EMA,p1_R1_TD_PCT_EMA,p2_R1_TD_PCT_EMA,p1_R1_SUB_ATT_EMA,p2_R1_SUB_ATT_EMA,p1_R1_REV_EMA,p2_R1_REV_EMA,p1_R1_CTRL_EMA,p2_R1_CTRL_EMA,p1_SIG_STR_PCT_DETAILED_EMA,p2_SIG_STR_PCT_DETAILED_EMA,p1_R1_SIG_STR_PCT_DETAILED_EMA,p2_R1_SIG_STR_PCT_DETAILED_EMA,p1_SIG_STR_LANDED_EMA,p2_SIG_STR_LANDED_EMA,p1_SIG_STR_ATTEMPTED_EMA,p2_SIG_STR_ATTEMPTED_EMA,p1_TOTAL_STR_LANDED_EMA,p2_TOTAL_STR_LANDED_EMA,p1_TOTAL_STR_ATTEMPTED_EMA,p2_TOTAL_STR_ATTEMPTED_EMA,p1_TD_LANDED_EMA,p2_TD_LANDED_EMA,p1_TD_ATTEMPTED_EMA,p2_TD_ATTEMPTED_EMA,p1_R1_SIG_STR_LANDED_EMA,p2_R1_SIG_STR_LANDED_EMA,p1_R1_SIG_STR_ATTEMPTED_EMA,p2_R1_SIG_STR_ATTEMPTED_EMA,p1_R1_TOTAL_STR_LANDED_EMA,p2_R1_TOTAL_STR_LANDED_EMA,p1_R1_TOTAL_STR_ATTEMPTED_EMA,p2_R1_TOTAL_STR_ATTEMPTED_EMA,p1_R1_TD_LANDED_EMA,p2_R1_TD_LANDED_EMA,p1_R1_TD_ATTEMPTED_EMA,p2_R1_TD_ATTEMPTED_EMA,p1_HEAD_LANDED_EMA,p2_HEAD_LANDED_EMA,p1_HEAD_ATTEMPTED_EMA,p2_HEAD_ATTEMPTED_EMA,p1_BODY_LANDED_EMA,p2_BODY_LANDED_EMA,p1_BODY_ATTEMPTED_EMA,p2_BODY_ATTEMPTED